# 🎯 앙상블 검색기(Ensemble Retriever) 완전 정복

## 📚 개요

**앙상블 검색기(Ensemble Retriever)**는 여러 검색 알고리즘을 **똑똑하게 조합**해서 더 강력한 검색 결과를 만들어내는 LangChain의 핵심 기능입니다! 🚀

마치 **여러 전문가의 의견을 종합해서 최종 결론을 내리는** 것과 같은 방식으로 동작합니다.

### 🎭 실생활 비유로 이해하기

**도서관에서 책을 찾는 상황**을 떠올려보세요:

- **📖 사서 A (Sparse Retriever)**: "키워드로 찾아드릴게요! '파이썬', '머신러닝' 같은 단어가 포함된 책들이에요"
- **🧠 사서 B (Dense Retriever)**: "의미를 파악해서 찾아드릴게요! '데이터 분석'과 유사한 개념의 책들이에요" 
- **👥 수석 사서 (Ensemble)**: "두 분의 추천을 종합해서 가장 적합한 책 순서로 정리해드릴게요!"

### 🎯 이 튜토리얼에서 배울 것들

1. **🔍 검색기 종류 이해하기** - Sparse vs Dense 검색의 차이점
2. **⚡ EnsembleRetriever 구현** - 여러 검색기를 하나로 묶는 방법
3. **🎛️ 가중치 조정** - 어떤 검색기를 더 중요하게 생각할지 설정
4. **🔧 런타임 설정 변경** - 실행 중에도 검색 전략 바꾸는 방법
5. **📊 성능 비교** - 단일 검색 vs 앙상블 검색 결과 비교

### 🏆 EnsembleRetriever의 핵심 특징

#### 1️⃣ **여러 검색기 통합** 🔗
- 다양한 유형의 검색기를 입력으로 받아 결과를 **똑똑하게 결합**
- BM25, FAISS, Chroma 등 어떤 검색기든 조합 가능

#### 2️⃣ **결과 재순위화** 📊  
- [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) 알고리즘 사용
- 각 검색기의 결과를 **공정하게** 순위 조정

#### 3️⃣ **하이브리드 검색** ⚡
- **Sparse Retriever** (BM25): 키워드 기반 검색에 강함 💪
- **Dense Retriever** (임베딩): 의미 기반 검색에 강함 🧠

### 💡 왜 앙상블이 더 좋을까?

✅ **상호 보완**: Sparse와 Dense의 장점을 동시에 활용  
✅ **높은 정확도**: 단일 검색기보다 더 정확한 결과  
✅ **유연성**: 상황에 맞게 가중치 조정 가능  
✅ **안정성**: 한 검색기가 실패해도 다른 검색기가 보완

자, 이제 실제 코드로 강력한 앙상블 검색기를 만들어봅시다! 🚀

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

---

## 🛠️ 환경 설정

앙상블 검색기를 체험하기 전에 필요한 환경을 준비해봅시다!

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

---

## 🔧 EnsembleRetriever 구현하기

### 🏗️ 앙상블 검색기 만들기

이제 **두 개의 검색 전문가**를 고용해서 **최강 검색팀**을 만들어봅시다! 

#### 🎯 팀 구성
- **📊 BM25 검색기**: 키워드 매칭의 달인 (가중치 0.7)
- **🧠 FAISS 검색기**: 의미 이해의 전문가 (가중치 0.3)
- **👥 앙상블 매니저**: 두 전문가의 결과를 종합해서 최종 순위 결정

`EnsembleRetriever`를 초기화하여 `BM25Retriever`와 `FAISS` 검색기를 결합합니다. 각 검색기의 **가중치** 를 설정하여 어떤 검색 방식을 더 중요하게 여길지 결정할 수 있습니다.

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 샘플 문서 리스트 - Apple 관련 다양한 문서들
doc_list = [
    "I like apples",
    "I like apple company", 
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]

# BM25 검색기 초기화 - 키워드 기반 검색의 달인
bm25_retriever = BM25Retriever.from_texts(
    doc_list,  # 문서 리스트를 입력
)
bm25_retriever.k = 1  # BM25 검색 결과 개수를 1로 제한

# OpenAI 임베딩 모델 초기화 - 의미를 벡터로 변환하는 도구
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# FAISS 벡터 저장소 생성 - 의미 기반 검색의 전문가
faiss_vectorstore = FAISS.from_texts(
    doc_list,    # 문서 리스트
    embedding,   # 임베딩 모델
)
# FAISS를 검색기로 변환 (검색 결과 1개로 제한)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 🎯 앙상블 검색기 초기화 - 두 전문가를 하나로 묶기!
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],  # 두 검색기를 리스트로 입력
    weights=[0.7, 0.3],  # BM25:70%, FAISS:30% 가중치 설정
)

### 🔍 검색 성능 비교하기

이제 **세 가지 검색 방식**의 결과를 직접 비교해봅시다! 

- **🎯 앙상블 검색**: 두 전문가의 의견을 종합한 최종 결과
- **📊 BM25 단독**: 키워드 매칭만으로 찾은 결과  
- **🧠 FAISS 단독**: 의미 유사도만으로 찾은 결과

각각의 **검색 전략** 이 어떻게 다른 결과를 내는지 확인해볼까요?

In [ ]:
# 검색 쿼리 정의 - "내가 좋아하는 과일은 사과다"
query = "my favorite fruit is apple"

# 세 가지 검색 방법으로 동일한 쿼리 검색
ensemble_result = ensemble_retriever.invoke(query)  # 앙상블 검색 (종합 판단)
bm25_result = bm25_retriever.invoke(query)          # BM25 검색 (키워드 매칭)
faiss_result = faiss_retriever.invoke(query)        # FAISS 검색 (의미 유사도)

# 🎯 앙상블 검색 결과 출력
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

# 📊 BM25 검색 결과 출력 
print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

# 🧠 FAISS 검색 결과 출력
print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

In [ ]:
# 다른 검색 쿼리로 테스트 - "Apple 회사가 내가 좋아하는 아이폰을 만든다"
query = "Apple company makes my favorite iphone"

# 동일한 세 가지 방법으로 새로운 쿼리 검색 
ensemble_result = ensemble_retriever.invoke(query)  # 앙상블 검색 결과
bm25_result = bm25_retriever.invoke(query)          # BM25 단독 검색 결과  
faiss_result = faiss_retriever.invoke(query)        # FAISS 단독 검색 결과

# 🎯 앙상블 검색 결과 - 두 검색기의 종합 판단
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

# 📊 BM25 검색 결과 - 키워드 "Apple", "company", "iphone" 매칭 우선
print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

# 🧠 FAISS 검색 결과 - 의미적 유사성 우선 (회사, 제품에 대한 선호)
print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

---

## 🎛️ 런타임 Config 변경 - 실시간 전략 조정

### 🔧 동적 설정의 힘

**런타임** 에서도 검색기의 **가중치** 를 자유자재로 변경할 수 있습니다! 이는 `ConfigurableField` 클래스의 **마법** 덕분이죠. ✨

#### 💡 실생활 비유
마치 **자동차의 운전 모드**를 바꾸는 것과 같습니다:
- **스포츠 모드** 🏎️: BM25에 더 높은 가중치 (정확한 키워드 매칭 중시)  
- **에코 모드** 🌱: FAISS에 더 높은 가중치 (의미적 유사성 중시)
- **혼합 모드** ⚖️: 균형 잡힌 가중치 배분

### 🎯 ConfigurableField의 장점

✅ **유연성**: 실행 중에도 전략 변경 가능  
✅ **실험 용이**: 다양한 가중치 조합을 빠르게 테스트  
✅ **맞춤화**: 상황에 맞는 최적 설정 적용  
✅ **재사용성**: 한 번 설정하면 다양한 시나리오에서 활용

### ⚙️ ConfigurableField 설정하기

`weights` 매개변수를 `ConfigurableField` 객체로 정의하여 **실행 시점** 에 가중치를 자유롭게 조정할 수 있게 만들어봅시다.

- 필드의 **ID** 는 "ensemble_weights"로 설정합니다.
- 이 ID를 통해 나중에 **config** 매개변수로 가중치를 변경할 수 있습니다!

In [ ]:
from langchain_core.runnables import ConfigurableField

# 실행 시점에 가중치를 변경할 수 있는 앙상블 검색기 생성
ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록 설정 - BM25와 FAISS 검색기 조합
    retrievers=[bm25_retriever, faiss_retriever],
).configurable_fields(
    # weights 매개변수를 실행 시점에 설정 가능하도록 구성
    weights=ConfigurableField(
        id="ensemble_weights",              # 설정 ID - config에서 이 이름으로 참조
        name="Ensemble Weights",           # 사람이 읽기 쉬운 이름
        description="Ensemble Weights",    # 설정에 대한 설명
    )
)

### 🏎️ BM25 우선 모드 (스포츠 모드)

검색 시 `config` 매개변수를 통해 **검색 전략** 을 실시간으로 지정합니다.

- `ensemble_weights` 옵션의 가중치를 `[1, 0]`으로 설정
- **모든 검색 결과의 가중치가 BM25 retriever 에 100% 집중** 💪
- **키워드 정확 매칭** 을 최우선으로 하는 전략입니다!

In [ ]:
# 설정 객체 생성 - BM25 검색기에만 100% 가중치 부여 (키워드 매칭 우선)
config = {"configurable": {"ensemble_weights": [1, 0]}}

# config 매개변수로 BM25 우선 모드로 검색 실행
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # BM25 검색기만을 사용한 결과 확인

### 🌱 FAISS 우선 모드 (에코 모드)

이번에는 검색 전략을 **의미적 유사성 중심** 으로 바꿔봅시다!

- 가중치를 `[0, 1]`로 설정하여 **FAISS retriever 에 100% 집중** 🧠  
- **문맥과 의미** 를 우선으로 하는 더욱 **똑똑한 검색** 전략입니다!

In [ ]:
# 설정 객체 생성 - FAISS 검색기에만 100% 가중치 부여 (의미 유사성 우선)  
config = {"configurable": {"ensemble_weights": [0, 1]}}

# config 매개변수로 FAISS 우선 모드로 검색 실행
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # FAISS 검색기만을 사용한 결과 확인